<a href="https://colab.research.google.com/github/josecal2844/Algoritmos-frecuentes/blob/main/Arquitectura_en_Keras_del_art%C3%ADculo_%22Road_Extraction_by_Deep_Residual_U_Net%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El presente trabajo consiste en la implementación de la arquitectura de una red neuronal profunda propuesta en el artículo "Road Extraction by Deep Residual U-Net", cuyo objetivo es la segmentación de carreteras en imágenes aéreas de alta resolución.

La arquitectura propuesta en el paper, llamada ResUnet, es una combinación entre dos modelos populares:

U-Net, conocida por su eficacia en tareas de segmentación de imágenes, especialmente en contextos con pocos datos.

ResNet (Red Residual), que introduce "conexiones residuales" para permitir el entrenamiento de redes más profundas sin que se pierda información por problemas como el desvanecimiento del gradiente.

### 1. Importar las librerías necesarias

In [1]:
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation, Add,
    UpSampling2D, Concatenate
)
from tensorflow.keras.models import Model

### 2. Creación del bloque residual

In [2]:
# Definición del bloque residual
def residual_unit(x, filters, stride=1):
    shortcut = x  # Guardamos la entrada original (para la suma residual)

    # Primera convolución
    x = BatchNormalization()(x)  # Normaliza las activaciones
    x = Activation('relu')(x)    # Aplica ReLU
    x = Conv2D(filters, (3, 3), strides=stride, padding='same')(x)  # Convolución 3x3

    # Segunda convolución
    x = BatchNormalization()(x)  # Normaliza nuevamente
    x = Activation('relu')(x)    # Aplica otra ReLU
    x = Conv2D(filters, (3, 3), strides=1, padding='same')(x)  # Segunda convolución 3x3

    # Si hay un cambio en la dimensionalidad, ajustamos el shortcut
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), strides=stride, padding='same')(shortcut)

    # Suma residual: agregamos la entrada original al resultado final
    x = Add()([x, shortcut])
    return x

### 3. Construir el modelo completo ResUnet

In [3]:
# Construcción del modelo ResUnet
def build_resunet(input_shape=(224, 224, 3)):
    inputs = Input(input_shape)

    # === ENCODER ===
    x1 = residual_unit(inputs, 64, stride=1)   # Nivel 1
    x2 = residual_unit(x1, 128, stride=2)      # Nivel 2
    x3 = residual_unit(x2, 256, stride=2)      # Nivel 3

    # === BRIDGE ===
    b = residual_unit(x3, 512, stride=2)       # Nivel 4

    # === DECODER ===
    d1 = UpSampling2D()(b)                     # Aumentamos el tamaño
    d1 = Concatenate()([d1, x3])               # Unimos con info del encoder
    d1 = residual_unit(d1, 256)

    d2 = UpSampling2D()(d1)
    d2 = Concatenate()([d2, x2])
    d2 = residual_unit(d2, 128)

    d3 = UpSampling2D()(d2)
    d3 = Concatenate()([d3, x1])
    d3 = residual_unit(d3, 64)

    # === SALIDA ===
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(d3)

    # Definimos el modelo final
    model = Model(inputs, outputs)
    return model

In [5]:
# Crear el modelo
model = build_resunet()

# Compilación del modelo con la función de error MSE
model.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy'])

# Ver un resumen del modelo
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │         12 │ input_layer_1[0]… │
│ (BatchNormalizatio… │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_14       │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 224, 224,  │      1,792 │ activation_14[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ conv2d_22[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_15       │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 224, 224,  │     36,928 │ activation_15[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 224, 224,  │        256 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 224, 224,  │          0 │ conv2d_23[0][0],  │
│                     │ 64)               │            │ conv2d_24[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ add_7[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_16       │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 112, 112,  │     73,856 │ activation_16[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_25[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_17       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_26 (Conv2D)  │ (None, 112, 112,  │    147,584 │ activation_17[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_27 (Conv2D)  │ (None, 112, 112,  │      8,320 │ add_7[0][0]       │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 112, 112,  │          0 │ conv2d_26[0][0],

 Total params: 8,227,405 (31.39 MB)

 Trainable params: 8,220,999 (31.36 MB)

 Non-trainable params: 6,406 (25.02 KB)